In [2]:
!pip install langchain_community
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 7.8 MB/s eta 0:00:00


In [12]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [5]:
!pip install pymupdf
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 104.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 75.8 MB/s eta 0:00:00


In [8]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("20250821_company_463434000.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 7


In [9]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 17


In [13]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [16]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [17]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [19]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("이 기업의 시가총액은 얼마야?")

[Document(id='be39b78a-68fa-4806-aa3d-f46b375803eb', metadata={'producer': 'Microsoft® Word Microsoft 365용', 'creator': 'Microsoft® Word Microsoft 365용', 'creationdate': '2025-08-20T21:43:04+09:00', 'source': '20250821_company_463434000.pdf', 'file_path': '20250821_company_463434000.pdf', 'total_pages': 7, 'format': 'PDF 1.7', 'title': '', 'author': 'vdiadmin', 'subject': '', 'keywords': '', 'moddate': '2025-08-20T21:43:04+09:00', 'trapped': '', 'modDate': "D:20250820214304+09'00'", 'creationDate': "D:20250820214304+09'00'", 'page': 0}, page_content='KOSDAQ \n777.61 \n시가총액 \n3,518십억원 \n시가총액비중 \n0.93% \n자본금(보통주) \n7십억원 \n52주 최고/최저 \n70,700원 / 40,050원 \n120일 평균거래대금 \n276억원 \n외국인지분율 \n70.22% \n주요주주 \n BCPE Centur Investments, LP 외 2 인 \n66.95%  \n  \n \n \n*KOL(Key Opinion Leader): 해당 분야에서 전문성과 영향\n력을 갖추어 다른 의사, 소비자, 업계 종사자들의 의견과 \n의사결정에 큰 영향을 미치는 핵심 전문가 \n주가수익률(%) \n1M \n3M \n6M \n12M \n절대수익률 \n-8.0  \n-7.6  \n-3.4  \n7.4  \n상대수익률 \n-3.0  \n-14.9  \n-4.6  \n8.8  \n \n-10\n0\n10\n20\n30\n

In [20]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Question:
{question}
#Context:
{context}

#Answer:"""
)

In [21]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [22]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 시가총액은 얼마야?"
response = chain.invoke(question)
print(response)

이 기업의 시가총액은 3,518십억원입니다.


In [24]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 투자의견 알려줘."
response = chain.invoke(question)
print(response)

이 기업의 투자의견은 "매수"입니다.


In [26]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 현재주가와 목표주가 알려줘."
response = chain.invoke(question)
print(response)

현재 주가에 대한 정보는 제공되지 않았습니다. 그러나 목표 주가는 78,000원으로 유지되고 있습니다.


In [27]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "이 기업의 리포트 요약해줘."
response = chain.invoke(question)
print(response)

이 기업의 리포트에 따르면, 클래시스는 시장에 빠르게 침투하기 위해 론칭 심포지엄을 진행하고 있으며, 가시적인 성과를 기대하고 있습니다. 투자의견은 '매수'로, 목표 주가는 78,000원으로 유지되고 있습니다. 이는 12개월 선행 EPS 2,342원에 목표 PER 30.1배를 적용한 것입니다. 이루다 인수로 인한 단기 마진 희석 우려는 시너지 효과로 빠르게 해소되고 있으며, 현재 주가는 12개월 선행 PER 20배 수준으로 밸류에이션 매력도가 높아진 상황입니다. 유럽 시장 진출과 신제품 효과를 고려하면 성장 잠재력 대비 저평가된 구간으로 평가됩니다. 

2023년부터 2027년까지의 매출액과 영업이익은 각각 180억 원에서 575억 원, 90억 원에서 291억 원으로 증가할 것으로 예상됩니다. 영업이익률은 약 50% 수준을 유지할 것으로 보입니다. 또한, 이루다 합병이 2024년 4분기부터 반영될 예정입니다.
